In [ ]:
import re
import requests
from bs4 import BeautifulSoup
MIN_ID = 1001
MAX_ID = 4202
DATABASE_NAME = 'Hemolytik'

In [ ]:
url_base = 'http://crdd.osdd.net/raghava/hemolytik/display.php?details='
amps = {}

In [ ]:
for j in range(MIN_ID + len(amps), MAX_ID):
# for j in range(1098, 1100):
    url = url_base + str(j)
    content = requests.get(url).content
    soup = BeautifulSoup(content, 'html.parser')
    rows = soup.findAll('td')
    sequence, hemolysis_activity, reference_article, reference_authors, reference_journal = None, None, None, None, None
    cterm_string, nterm_string, lc_string, stereochemistry_string, nn_string = None, None, None, None, None
    for i, row in enumerate(rows):
        string = row.string
        if row.string:
            stripped = row.string.strip()
            if stripped == 'SEQUENCE':
                sequence = rows[i+1].string.strip().replace('-', '')
            elif stripped == 'ACTIVITY':
                hemolysis_activity = rows[i+1].string.strip()
            elif stripped == 'ARTICLE':
                reference_article = rows[i+1].string.strip()
            elif stripped == 'AUTHORS':
                reference_authors = rows[i+1].string.strip() if rows[i+1].string else ''
            elif stripped == 'JOURNAL':
                reference_journal = rows[i+1].string.strip()
            elif stripped == 'C-ter Modification':
                cterm_string = rows[i+1].string.strip()
            elif stripped == 'N-ter Modification':
                nterm_string = rows[i+1].string.strip()
            elif stripped == 'Linear/Cyclic':
                lc_string = rows[i+1].string.strip()
            elif stripped == 'Stereochemistry':
                stereochemistry_string = rows[i+1].string.strip()
            elif stripped == 'Non-Natural':
                nn_string = rows[i+1].string.strip()

    try:
        references = [reference_article + " " + reference_authors + " " + reference_journal]
    except:
        continue  # A page like http://crdd.osdd.net/raghava/hemolytik/display.php?details=1026

    modifications = []
    if 'Amidat' in cterm_string:
        modifications.append('C-Terminal')
    elif cterm_string != 'Free':
        modifications.append('C-Terminal Non-Amidation: ' + cterm_string)
    if nterm_string != 'Free':
        modifications.append('N-Terminal Modification: ' + nterm_string)
    if lc_string != 'Linear':
        modifications.append('Cyclic')
    if stereochemistry_string != 'L':
        modifications.append('Stereochemistry: ' + stereochemistry_string)
    if nn_string != "None":
        modifications.append('Non-Natural')

    amps[sequence] = {
        'hemolysis': hemolysis_activity,
        'url_sources': [url],
        'references': references,
        'modifications': modifications
    }
    if j % 100 == 0:
        print(j)

In [ ]:
with open(DATABASE_NAME + ".data", 'w') as f:
    f.write(str(amps))